In [ ]:
!pip install -U https://github.com/conda-incubator/condacolab/archive/cuda-version-12.tar.gz
import condacolab
condacolab.install_mambaforge()

In [ ]:
!wget -qN https://raw.githubusercontent.com/openforcefield/openff-docs/main/source/workshops/2024/vignettes/utils.py
!wget -qN https://raw.githubusercontent.com/openforcefield/openff-docs/main/source/workshops/2024/vignettes/colab-env.yml
!wget -qN https://raw.githubusercontent.com/openforcefield/openff-docs/main/source/workshops/2024/vignettes/micelle-self-assembly-backup.xtc
!wget -qN https://raw.githubusercontent.com/openforcefield/openff-docs/main/source/workshops/2024/vignettes/micelle-self-assembly-start-backup.pdb
!wget -qN -P inputs https://raw.githubusercontent.com/openforcefield/openff-docs/main/source/workshops/2024/vignettes/inputs/3ip9_dye_solvated.pdb
!wget -qN -P inputs https://raw.githubusercontent.com/openforcefield/openff-docs/main/source/workshops/2024/vignettes/inputs/dlpc.sdf
!wget -qN -P inputs https://raw.githubusercontent.com/openforcefield/openff-docs/main/source/workshops/2024/vignettes/inputs/example-vsites-parameters-forcefield.offxml
!wget -qN -P inputs https://raw.githubusercontent.com/openforcefield/openff-docs/main/source/workshops/2024/vignettes/inputs/PT2385.sdf
!wget -qN -P inputs https://raw.githubusercontent.com/openforcefield/openff-docs/main/source/workshops/2024/vignettes/inputs/solvated_complex.pdb
!wget -qN -P inputs https://raw.githubusercontent.com/openforcefield/openff-docs/main/source/workshops/2024/vignettes/inputs/ribozymes.pdb
!mamba env update -q --name=base --file=colab-env.yml

In [ ]:
from google.colab import output
output.enable_custom_widget_manager()

# Basic Simulation from PDB

The starting configuration for this vignette comes from the Toolkit Showcase, which assembles and solvates this topology starting from a prepared protein PDB and ligand SDF. 

https://github.com/openforcefield/openff-toolkit/tree/main/examples/toolkit_showcase

This notebook describes the basic workflow for any OpenFF simulation.

## OpenFF Code

In [ ]:
from openff.toolkit import ForceField, Molecule, Topology

ligand = Molecule.from_file("inputs/PT2385.sdf")
top = Topology.from_pdb("inputs/solvated_complex.pdb", unique_molecules=[ligand])
ff = ForceField("openff-2.2.0.offxml", "ff14sb_off_impropers_0.0.4.offxml")

ic = ff.create_interchange(top)
ic.visualize()

## OpenMM Code

In [ ]:
import openmm
from openff.units import Quantity, unit

simulation = ic.to_openmm_simulation(
    openmm.LangevinIntegrator(
        300 * openmm.unit.kelvin,
        1 / openmm.unit.picosecond,
        2 * openmm.unit.femtoseconds,
    )
)

# Energy minimize
simulation.minimizeEnergy()

# Add a reporter to record the structure every 1000 steps
dcd_reporter = openmm.app.DCDReporter("trajectory.dcd", 1000)
simulation.reporters.append(dcd_reporter)
simulation.context.setVelocitiesToTemperature(simulation.integrator.getTemperature())
simulation.runForClockTime(1 * openmm.unit.minute)

## NGLView

In [ ]:
import mdtraj
import nglview

trajectory: mdtraj.Trajectory = mdtraj.load(
    "trajectory.dcd", top=mdtraj.Topology.from_openmm(top.to_openmm())
)

view = nglview.show_mdtraj(trajectory.image_molecules())
view.add_representation("line", selection="water")
view.add_representation(
    "hyperball", radiusSize=1, radiusScale=0.5, selection="not protein and not water"
)
view

# Virtual Sites

This notebook describes the use of virtual sites in OpenFF force fields.

In [ ]:
from openff.toolkit import ForceField

sage_with_example_virtual_sites = ForceField(
    "openff-2.2.0.offxml",
    "inputs/example-vsites-parameters-forcefield.offxml",
)

In [ ]:
import mdtraj
import nglview
import openmm
import openmm.unit
from openff.toolkit import Molecule

def _add_vsite_reps(widget, interchange):
    widget.clear_representations()
    atoms_with_vsites = [*interchange.to_openmm_topology().atoms()]
    if len(atoms_with_vsites) == interchange.topology.n_atoms + 1:
        vsite = next(iter(interchange["VirtualSites"].key_map))
    
        atom1 = vsite.orientation_atom_indices[0]
        name = vsite.name
    
        atom2 = [atom.index for atom in atoms_with_vsites if atom.name == name][0]
    
        widget.add_distance(atomPair=[[f"@{atom1}", f"@{atom2}"]])

    vsite_idcs = [
        str(v.index) for v in atoms_with_vsites[interchange.topology.n_atoms:]
    ]
    not_vsite_idcs = [
        str(v.index) for v in atoms_with_vsites[:interchange.topology.n_atoms]
    ]

    widget.add_licorice(selection="@" + ",".join(not_vsite_idcs))
    widget.add_representation("ball+stick", selection="@" + ",".join(vsite_idcs))

    return widget
        
def viz(
    smiles: str,
    force_field: ForceField = sage_with_example_virtual_sites,
) -> nglview.NGLWidget:
    molecule = Molecule.from_smiles(smiles, allow_undefined_stereo=True)
    molecule.generate_conformers(n_conformers=1)

    interchange = force_field.create_interchange(molecule.to_topology())
    
    w = interchange.visualize(backend='nglview', include_virtual_sites=True)
    return _add_vsite_reps(w, interchange)


def run(
    smiles: str,
    name: str,
    force_field: ForceField = sage_with_example_virtual_sites,
) -> nglview.NGLWidget:
    molecule = Molecule.from_smiles(smiles, allow_undefined_stereo=True)
    molecule.generate_conformers(n_conformers=1)

    interchange = force_field.create_interchange(molecule.to_topology())

    integrator = openmm.LangevinIntegrator(
        300 * openmm.unit.kelvin,
        1 / openmm.unit.picosecond,
        1 * openmm.unit.femtoseconds,
    )

    simulation = interchange.to_openmm_simulation(integrator=integrator)

    dcd_reporter = openmm.app.DCDReporter("trajectory.dcd", 10)
    simulation.reporters.append(dcd_reporter)

    simulation.context.computeVirtualSites()
    simulation.minimizeEnergy()
    simulation.context.setVelocitiesToTemperature(integrator.getTemperature())
    simulation.step(1000)

    interchange.positions = simulation.context.getState(
        getPositions=True
    ).getPositions()

    open(f"{name}.xml", "w").write(
        openmm.XmlSerializer.serialize(interchange.to_openmm())
    )

    w = nglview.show_mdtraj(
        mdtraj.load(
            "trajectory.dcd",
            top=mdtraj.Topology.from_openmm(
                interchange.to_openmm_topology(),
            ),
        )
    )
    return _add_vsite_reps(w, interchange)


## `BondCharge` virtual site

This supports placement of a virtual site S along a vector between two specified atoms, e.g. to allow for a sigma hole for halogens or similar contexts. With positive values of the distance, the virtual site lies outside the first indexed atom (cyan in this image).

![](https://raw.githubusercontent.com/openforcefield/openff-docs/main/source/workshops/2024/vignettes/images/vsite_bondcharge.jpg)

In [ ]:
sage_with_example_virtual_sites["VirtualSites"].get_parameter({"type": "BondCharge"})[
    0
].to_dict()

In [ ]:
run("CCCCCl", name="sigma_hole")

## `MonovalentLonePair` virtual site

This is originally intended for situations like a carbonyl, and allows placement of a virtual site `S` at a specified distance `d` from the first indexed atom, with angles `inPlaneAngle` (θ₁ in the diagram) and `outOfPlaneAngle` (θ₂ in the diagram) relative to the plane defined by the three indexed atoms. The atom labeled `w` is not used to place the virtual site.

![](https://raw.githubusercontent.com/openforcefield/openff-docs/main/source/workshops/2024/vignettes/images/vsite_monovalent.jpg)

In [ ]:
sage_with_example_virtual_sites["VirtualSites"].get_parameter(
    {"type": "MonovalentLonePair"}
)[0].to_dict()

In [ ]:
run("COC1=C(C=CC(=C1)C=O)O", name="planar_carbonyl")

## `DivalentLonePair` virtual site

This is suitable for cases like four-point and five-point water models as well as pyrimidine; a virtual site `S` lies a specified distance `d` from the atom indexed 1 (blue) along the bisector of the angle between the three indexed atoms (if `outOfPlaneAngle` is zero) or out of the plane by the specified angle (if `outOfPlaneAngle` is nonzero) with its projection along the bisector. For positive values of the distance `d` the virtual site lies outside the 2-1-3 angle and for negative values it lies inside.

![](https://raw.githubusercontent.com/openforcefield/openff-docs/main/source/workshops/2024/vignettes/images/vsite_divalent.jpg)

In [ ]:
ForceField("tip4p_fb.offxml")["VirtualSites"].get_parameter({"type": "DivalentLonePair"})[
    0
].to_dict()


In [ ]:
run("O", name="tip4p_fb", force_field=ForceField("tip4p_fb.offxml"))

In [ ]:
ForceField("tip5p.offxml")["VirtualSites"].get_parameter({"type": "DivalentLonePair"})[
    0
].to_dict()


In [ ]:
run("O", name="tip5p", force_field=ForceField("tip5p.offxml"))

## `TrivalentLonePair` virtual site

This is suitable for planar or tetrahedral nitrogen lone pairs; a charge site `S` lies above the central atom (blue) a distance `d` along the vector perpendicular to the plane of the other three indexed atoms (2,3,4). With positive values of `d` the site lies above the central atom and with negative values it lies below.

![](https://raw.githubusercontent.com/openforcefield/openff-docs/main/source/workshops/2024/vignettes/images/vsite_trivalent.jpg)

In [ ]:
sage_with_example_virtual_sites["VirtualSites"].get_parameter(
    {"type": "TrivalentLonePair"}
)[0].to_dict()

In [ ]:
run("N", name="trivalent_nitrogen")

## Multiple virtual sites

Each of the molecules shown so far was selected to only have a single virtual site added, but larger molecules may have multiple copies of different types of virtual sites added.

In [ ]:
run("c1(Cl)c(Cl)c(CCCN)c(Cl)c(Cl)c1CCC(=O)", "many_vsites")

# Ligand modification with RDKit

This notebook demonstrates loading and modifying a molecule with RDKit.

## OpenFF Code

In [ ]:
from openff.toolkit import ForceField, Molecule, Topology

orig_ligand = Molecule.from_file("inputs/PT2385.sdf")
orig_top = Topology.from_pdb(
    "inputs/solvated_complex.pdb", unique_molecules=[orig_ligand]
)
ff = ForceField("openff-2.2.0.offxml", "ff14sb_off_impropers_0.0.4.offxml")

In [ ]:
orig_ligand.visualize(backend="rdkit")

## RDKit Code

In [ ]:
# Use RDKit's reaction handling to try mutating each aliphatic C-H bond to C-F
import rdkit

rdmol = orig_ligand.to_rdkit()
rxn = rdkit.Chem.rdChemReactions.ReactionFromSmarts("[C:1][H:2] >> [C:1][F:2]")
unsanitized_products = rxn.RunReactants([rdmol])

# Since some substitutions won't actually lead to a "new" molecule, only keep
# unique molecules
products = set()
for (product,) in unsanitized_products:
    mol_copy = rdkit.Chem.Mol(product)
    
    rdkit.Chem.SanitizeMol(mol_copy, rdkit.Chem.SANITIZE_ALL)
    rdkit.Chem.AssignStereochemistryFrom3D(mol_copy)
    rdkit.Chem.Kekulize(mol_copy, clearAromaticFlags=True)
    rdkit.Chem.SetAromaticity(mol_copy, rdkit.Chem.AromaticityModel.AROMATICITY_MDL)
    
    products.add(Molecule.from_rdkit(mol_copy))

In [ ]:
def display_molecule_grid(molecules, item_width=200):
    import ipywidgets as widgets
    
    items = []
    for product in products:
        item = widgets.Output()
        item.append_display_data(product.visualize('rdkit', width=item_width))
        items.append(item)
        
    return widgets.GridBox(
        items, 
        layout=widgets.Layout(grid_template_columns=f"repeat(3, {item_width+10}px)")
    )
display_molecule_grid(products)

## OpenFF and OpenMM Code

In [ ]:
import openmm
from openff.units import Quantity, unit
from openmm import unit as openmm_unit

idx_to_sim = dict()
n_systems = len(products)

for idx, unique_offmol in enumerate(products):
    print(f"Making topology {idx+1}/{n_systems}")

    # Create a new topology from the with all but the last molecule (the ligand)
    this_top = Topology.from_molecules([*orig_top.molecules][:-1] + [unique_offmol])
    this_top.box_vectors = orig_top.box_vectors
    pdb_filename = f"topology{idx+1}.pdb"
    this_top.to_file(pdb_filename)

    print(f"Parametrizing system {idx+1}/{n_systems}")
    integrator = openmm.LangevinIntegrator(
        300 * openmm_unit.kelvin,
        1 / openmm_unit.picosecond,
        0.002 * openmm_unit.picoseconds,
    )
    
    simulation = ff.create_interchange(this_top).to_openmm_simulation(integrator)

    print(f"Minimizing system {idx+1}/{n_systems}")
    simulation.minimizeEnergy()
    
    print(f"Simulating system {idx+1}/{n_systems}")
    traj_filename = f"trajectory_{idx+1}.dcd"
    dcd_reporter = openmm.app.DCDReporter(traj_filename, 50)
    simulation.reporters.append(dcd_reporter)
    simulation.context.setVelocitiesToTemperature(300 * openmm_unit.kelvin)
    simulation.runForClockTime(15 * openmm_unit.second)
    idx_to_sim[idx + 1] = {
        "topology": this_top,
        "pdb_filename": pdb_filename,
        "simulation": simulation,
        "trajectory": traj_filename,
    }

## NGLView

In [ ]:
import mdtraj
import nglview

SIM_TO_VISUALIZE = 1

trajectory: mdtraj.Trajectory = mdtraj.load(
    idx_to_sim[SIM_TO_VISUALIZE]["trajectory"],
    top=mdtraj.load(idx_to_sim[SIM_TO_VISUALIZE]["pdb_filename"]).topology,
)

view = nglview.show_mdtraj(trajectory.image_molecules())
view.add_representation("line", selection="water")
view.add_representation(
    "hyperball", radiusSize=1, radiusScale=0.5, selection="not protein and not water"
)
view

# Retrieving datasets from QCFractal with `openff-qcsubmit`

**Based on the existing example**: https://github.com/openforcefield/openff-qcsubmit/blob/main/examples/retrieving-results.ipynb

This example shows how QCSubmit can be used to retrieve the results of quantum chemical (QC) calculations from a [QCFractal] instance such as [QCArchive].

In particular, it demonstrates how:

* raw torsion drive, optimised geometry and hessian result records can be retrieved from the public
  [QCArchive] server and stored in a result collection

* the retrieved result records can be filtered and curated using a set of built-in filters

* the result collection can be saved and loaded from disk

[QCFractal]: http://docs.qcarchive.molssi.org/projects/qcfractal/en/latest/
[QCArchive]: https://qcarchive.molssi.org/

For the sake of clarity all verbose warnings will be disabled in this tutorial:

In [ ]:
import warnings

warnings.filterwarnings("ignore")
import logging

logging.getLogger("openff.toolkit").setLevel(logging.ERROR)

## Retrieving result collections

QCSubmit provides a suite of utilities for retrieving and curating collections of QC results directly from a running QCFractal server, or an already computed QCPortal dataset. This functionality is provided through three main classes:

* `BasicResultCollection` - stores references to simple QCPortal result record that may contain energies, gradients, or hessians computed for a molecule in a single conformation.

* `OptimizationResultCollection` - stores references to full optimization result records (i.e. `OptimizationRecord`
  objects), as well as the final minimised conformer produced by the optimization.

* `TorsionDriveResultCollection` - stores references to full torsion drive result records (i.e. `TorsionDriveRecord`
  objects), as well as the minimum energy conformer associated with each torsion angle that was scanned.

Each of these collections can be generated directly from a running `QCFractal` server using the `from_server` class
method.

We begin by creating a QCPortal `FractalClient` instance that will allow us to communicate with the running
server. By default, `FractalClient` connects to the main QCArchive server:

In [ ]:
from qcportal import PortalClient

qc_client = PortalClient("https://api.qcarchive.molssi.org:443")

Other servers can be accessed by providing the server's URI.

We can then use this to generate our result collections:

In [ ]:
from openff.qcsubmit.results import (
    BasicResultCollection,
    OptimizationResultCollection,
    TorsionDriveResultCollection,
)

# Pull down the torsion drive records from the 'OpenFF Protein Capped 3-mer Backbones v1.0' dataset.
torsion_drive_result_collection = TorsionDriveResultCollection.from_server(
    client=qc_client,
    datasets="OpenFF Protein Capped 3-mer Backbones v1.0",
    spec_name="default",
)

*Note: currently only complete results are pulled down by the `from_server` method*

There are two main inputs to the `from_server` method, in addition to the fractal client:

* the name(s) of the existing datasets to retrieve the results of. This can either be the name of a single dataset or a list of dataset names
* the name of the specification used to compute the records. Each specification corresponds to a particular basis, method, program and additional settings.

Let's print out some basic information about each of these result collections:

In [ ]:
print("===TORSION DRIVE RESULTS===")

print(f"N RESULTS:   {torsion_drive_result_collection.n_results}")
print(f"N MOLECULES: {torsion_drive_result_collection.n_molecules}")

This allows results generated by multiple different servers (e.g. a local fractal instance and the public QCArchive
server) to be stored in a single result collection object.

The references to the actual data are then stored in corresponding lists:

In [ ]:
torsion_drive_result_collection.entries[qc_client.address][:10]

After running the above command, notice that the entries stored in the collection are not the actual result
records generated and stored on the server, but rather a reference to them. In particular, the unique ID of the record is stored along with a SMILES depiction of the molecule the result was generated for.

The main reason for doing this is that we often would like to be able to state which data we would like to use in
an application without having to create multiple copies of the data. Not only can this take up large amounts of disk space, it runs the risk of data becoming out of sync with the original if the format the records are stored in changes or the local copy of the data is accidentally mutated. Storing a reference to the original data and then retrieving it when needed is typically a cleaner and safer solution.

## Retrieving the result records

The raw result record objects can be easily retrieved using the result collection objects. This allows us to filter the collection to only retrieve the results we want. For example, we can apply a SMARTS string to retrieve the cysteine record:

In [ ]:
from openff.qcsubmit.results.filters import SMARTSFilter

filtered_collection = torsion_drive_result_collection.filter(SMARTSFilter(smarts_to_include=["C[SH]"]))
filtered_collection

Then we can download the actual results. This can be very slow, so it's worth filtering aggressively:

In [ ]:
torsion_drive_records = filtered_collection.to_records()
torsion_drive_records

QCSubmit seamlessly takes care of pulling the data from the server in the most efficient way making sure to take
advantage of the pagination that QCFractal provides. Further, it attempts to cache all calls to the server so that
multiple calls to `to_records` does not need to constantly query the server.

Notice that not only are the raw result records retrieved, but also an OpenFF `Molecule` object is created for each result record. This molecule has the correct ordering and also stores any conformers associated with the
result collection. For basic collections, the conformer is the one that was used in any calculations; for optimization collections, it is the final conformer yielded by the optimization; and for torsion drives, it is the lowest energy conformer for each sampled torsion angle.

## Inspecting results

In the case of torsion drive records, we can easily iterate over the grid ID, the associated conformer, and the
associated energy in one go:

In [ ]:
torsion_drive_record, molecule = torsion_drive_records[0]
import numpy as np
from matplotlib import pyplot
from openff.units import unit

energy_grid = np.zeros((24, 24))
psi_labels = [""] * 24
phi_labels = [""] * 24
for (phi, psi), qc_conformer in zip(
    molecule.properties["grid_ids"], molecule.conformers
):
    qc_energy = torsion_drive_record.final_energies[(phi, psi)]

    phi_bin = int(phi + 165) // 15
    psi_bin = int(psi + 165) // 15
    energy_grid[psi_bin, phi_bin] = qc_energy
    psi_labels[psi_bin] = psi
    phi_labels[phi_bin] = phi
    # print(f"({phi}, {psi}) {phi_bin},{psi_bin} E={qc_energy:.4f} Ha")

In [ ]:
import plotly.graph_objects as go
from ipywidgets import widgets

fig = go.FigureWidget(
    data=go.Heatmap(
        z=energy_grid,
        x=phi_labels,
        y=psi_labels,
        colorbar={"title": "Energy (Ha)"},
        hovertemplate="phi: %{x}\npsi: %{y}\nenergy: %{z} Ha",
    ),
    layout=go.Layout(
        title="Val-Ala-Val - central backbone torsiondrive (Ha)",
        xaxis_title="Phi",
        yaxis_title="Psi",
        # autosize=False,
        yaxis_scaleanchor="x",
        xaxis_scaleanchor="y",
    ),
)

view = molecule.visualize("nglview")

def on_click(trace, points, selector):
    print(points)
    for x, y in points.point_inds:
        view.frame = x * 24 + y


heatmap = fig.data[0]
heatmap.on_click(on_click)

container = widgets.GridBox(
    [fig, view],
    # layout=widgets.Layout(grid_template_columns=f"repeat(2, {600}px)"),
)
container

# Vectorized Interchange representations

In [ ]:
import numpy
from openff.interchange import Interchange
from openff.toolkit import ForceField, Molecule
from rich.pretty import pprint

In [ ]:
sage = ForceField("openff_unconstrained-2.2.0.offxml")
molecule = Molecule.from_smiles(r"F\C=C/F")
interchange = Interchange.from_smirnoff(sage, [molecule])

pprint(interchange.collections.keys())

In [ ]:
molecule.visualize()

`Interchange`s from SMIRNOFF force fields contain collections for several different types of parameters. For simplicity, let's look at the bond collection.

In [ ]:
collection = interchange.collections["Bonds"]

`Collection`s store force field parameters and information about how they related to the topology they are applied to. In addition, they include some handy methods for transforming these to vectorized representations.

In [ ]:
# Parameters from the force field; one row per force field parameter

pprint(collection.get_force_field_parameters())

`Collection.get_force_field_parameters` returns an array with one row per unique force field parameter used and one colum per number in each parameter. For this molecule, that means three rows (C-F, C#C, and C-H chemistries) and two columns (`k` and `length`). This matrix scales with the number of unique force field parameters used so it will not generally scale with system size.

In [ ]:
# Parameters assigned to the system; one row per atom

pprint(collection.get_system_parameters())

`Collection.get_system_field_parameters` returns a similar array but with one row per bond in the topology, including duplicates. Since there are two C-H and two C-F bonds, those parameters each appear twice. This matrix scales with the size of the system (in this case, number of bonds) so it may be large for large systems.

In [ ]:
# Mapping from atoms to force field parameters; one row per value per atom
# bonds have lengths and force constants, so two rows per atom in this case

pprint(collection.get_param_matrix())

It may be useful to encode the relationships between force field parameters and where in the topology they're applied. This is handled by `collection.get_param_matrix()`, which returns a spare matrix. Each column corresponds to a force field parameter and each row corresponds to a bond that could be associated with each, each dimension being a flattened representation of the above matrices. A 1 indicates that a parameter is applied to that bond, a 0 indicates that it is not. For example, the 1 at `[0, 0]` indicates that the first bond gets assigned the first `k` value. The 1 at `[7, 1]` indicates that the fourth bond gets assigned the first `length`.

Conveniently, the dot product of this matrix with a flattened view of the force field parameters is equal to the view of the system parameters we saw above.

In [ ]:
dotted = numpy.dot(
    interchange["Bonds"].get_param_matrix(),
    interchange["Bonds"].get_force_field_parameters().flatten(),
).reshape((-1, 2))

assert numpy.allclose(dotted, collection.get_system_parameters())

pprint(dotted)

# Lipid Micelle Self-assembly

This notebook prepares a DLPC micelle self assembly simulation.

This notebook is YELLOW because:

- Sage is not validated for lipids
- `pack_box` is an experimental private function

In [ ]:
from io import StringIO

import mdtraj
import nglview
import numpy as np
import openmm
from openff.interchange import Interchange
from openff.interchange.components._packmol import pack_box
from openff.toolkit import ForceField, Molecule, Topology
from openff.units import unit

In [ ]:
dlpc = Molecule.from_file("inputs/dlpc.sdf")
lipids = [dlpc]

conc_nacl = 0.1 * unit.mole / unit.liter
n_waters = 8000
n_lipids = [25]
target_density = 1.0 * unit.gram / unit.milliliter

In [ ]:
water = Molecule.from_smiles("O")
na = Molecule.from_smiles("[Na+]")
cl = Molecule.from_smiles("[Cl-]")

for atom in water.atoms:
    atom.metadata["residue_name"] = "HOH"

for atom in na.atoms:
    atom.metadata["residue_name"] = "NA+"

for atom in cl.atoms:
    atom.metadata["residue_name"] = "CL-"

molarity_pure_water = 55.5 * unit.mole / unit.liter
n_nacl = int((n_waters / molarity_pure_water * conc_nacl).to(unit.dimensionless).m)
molecules = [*lipids, water, na, cl]
n_copies = [*n_lipids, n_waters, n_nacl, n_nacl]

total_mass = sum(
    [
        sum([atom.mass for atom in molecule.atoms]) * n
        for molecule, n in zip(molecules, n_copies)
    ]
)
target_volume = (total_mass / target_density).to(unit.nanometer**3)
box_vectors = np.asarray([[1, 0, 0], [0, 1, 0], [0, 0, 1]]) * np.cbrt(target_volume)

top = pack_box(
    molecules,
    n_copies,
    box_vectors=box_vectors,
    tolerance=0.05 * unit.nanometer,
)

In [ ]:
top.to_file("micelle-self-assembly-start.pdb")

In [ ]:
w = top.visualize()
w.add_representation("line", selection="water")
w

In [ ]:
sage = ForceField("openff-2.2.0.offxml")

In [ ]:
interchange = Interchange.from_smirnoff(sage, top, charge_from_molecules=[dlpc])

In [ ]:
simulation = interchange.to_openmm_simulation(
    integrator=openmm.LangevinMiddleIntegrator(
        300 * openmm.unit.kelvin,
        1 / openmm.unit.picosecond,
        2 * openmm.unit.femtosecond,
    )
)

In [ ]:
simulation.minimizeEnergy()
minimized_state = simulation.context.getState(
    getPositions=True, getEnergy=True, getForces=True
)

print(
    "Minimised to",
    minimized_state.getPotentialEnergy(),
    "with maximum force",
    max(
        np.sqrt(v.x * v.x + v.y * v.y + v.z * v.z) for v in minimized_state.getForces()
    ),
    minimized_state.getForces().unit.get_symbol(),
)

minimized_coords = minimized_state.getPositions()

In [ ]:
simulation.system.addForce(
    openmm.MonteCarloBarostat(
        1.0 * openmm.unit.bar,
        simulation.integrator.getTemperature(),
    )
)
simulation.context.reinitialize(preserveState=True)

In [ ]:
# Add a reporter to record the structure every data_freq steps
data_freq = 10000
dcd_reporter = openmm.app.XTCReporter("micelle-self-assembly.xtc", data_freq)
simulation.reporters.append(dcd_reporter)

state_data_reporter = openmm.app.StateDataReporter(
    "data.csv",
    data_freq,
    step=True,
    potentialEnergy=True,
    kineticEnergy=True,
    temperature=True,
    density=True,
    elapsedTime=True,
    speed=True,
)
simulation.reporters.append(state_data_reporter)

In [ ]:
# simulation_time = 10 * unit.nanosecond
# steps = round((simulation_time / delta_t).to(unit.dimensionless).m)
# simulation.step(steps)

## Visualisation

In [ ]:
import mdtraj
import nglview

trajectory: mdtraj.Trajectory = mdtraj.load(
    # "micelle-self-assembly.xtc", 
    "micelle-self-assembly-backup.xtc", 
    # top=mdtraj.load("micelle-self-assembly-start.pdb").top, 
    top=mdtraj.load("micelle-self-assembly-start-backup.pdb").top, 
    stride=10,
)

# trajectory.image_molecules()

view = nglview.show_mdtraj(trajectory)
view.clear()
view.add_representation(
    "spacefill",
    selection=f"HOH NA+ CL-",
    opacity=0.1,
)
view.add_representation("licorice", selection="NOT (HOH NA+ CL-)")
view.add_unitcell()
view

# RNA simulation from OpenMM force field

This notebook is YELLOW because:
- Sage has not been validated to work with Amber force fields and has a different partial charge philosophy
- Many of the functions used are experimental

In [ ]:
%env INTERCHANGE_EXPERIMENTAL=1

## Load PDB

https://www.ncbi.nlm.nih.gov/pmc/articles/PMC3158920

In [ ]:
def molecule_from_rna_sequence(sequence):
    import random

    from openff.toolkit import Molecule
    from rdkit import Chem
    from rdkit.Chem import AllChem

    rna = Chem.MolFromSequence(sequence, flavor=2)

    for atom in rna.GetAtoms():
        if atom.GetNumImplicitHs() == 1 and [
            a.GetSymbol() for a in atom.GetNeighbors()
        ] == ["P"]:
            atom.SetFormalCharge(-1)

    Chem.SanitizeMol(rna)
    offmol = Molecule.from_rdkit(
        rna,
        allow_undefined_stereo=True,
    )

    return offmol


ribozyme = molecule_from_rna_sequence("GUGGC")
rna_substrate = molecule_from_rna_sequence("GCCU")

In [ ]:
from openff.toolkit import ForceField, Molecule, Topology

# PheAMP, phenylalanine adenosine monophosphate
pheamp = Molecule.from_smiles(
    "c1ccccc1C[C@H](N)C(=O)O[P@]([O-])(=O)OC[C@@H]2[C@@H](O)[C@@H](O)[C@@H](O2)N3C4N=CN=C(N)C=4N=C3",
    allow_undefined_stereo=False,
)
pheamp.visualize("rdkit", show_all_hydrogens=False)

In [ ]:
from openff.toolkit import Topology

top = Topology.from_pdb(
    "inputs/ribozymes.pdb",
    unique_molecules=[pheamp, ribozyme, rna_substrate],
)
top.visualize()

## Split topologies

In [ ]:
molecules = list(top.molecules)

rna_top = Topology.from_molecules(molecules[:2])
rna_top.box_vectors = top.box_vectors

sage_top = Topology.from_molecules(molecules[2:])
sage_top.box_vectors = top.box_vectors

In [ ]:
sage_top.visualize()

## Parametrize

In [ ]:
from openff.toolkit import ForceField

sage = ForceField("openff-2.2.0.offxml")
sage_interchange = sage.create_interchange(sage_top)

In [ ]:
from tempfile import NamedTemporaryFile

import numpy as np
import openmm.app
from openff.interchange import Interchange
from openff.toolkit import Molecule, Topology
from rdkit import Chem

openmm_force_field=openmm.app.ForceField(
    # "amber14/protein.ff14SB.xml",
    "amber14/RNA.OL3.xml",
    # "amber14/DNA.OL15.xml",
    # "amber14/lipid17.xml",
    # "amber14/GLYCAM_06j-1.xml",
    # "amber14/tip3p.xml",
)

openmm_system = openmm_force_field.createSystem(
    rna_top.to_openmm(),
    nonbondedMethod=openmm.app.PME,
    nonbondedCutoff=9 * openmm.unit.angstrom,
    switchDistance=8 * openmm.unit.angstrom,
    constraints=openmm.app.HBonds,
    rigidWater=True,
)

# Create the Interchange
rna_interchange = Interchange.from_openmm(
    topology=rna_top,
    system=openmm_system,
    positions=rna_top.get_positions(),
)

rna_interchange.visualize()

## Combine Interchanges

In [ ]:
# Work around a bug in Interchange 0.3.26
rna_interchange['Electrostatics'].cutoff = rna_interchange['vdW'].cutoff

solvated_interchange = sage_interchange.combine(rna_interchange)

In [ ]:
solvated_interchange.visualize()

## Simulate!

In [ ]:
simulation = solvated_interchange.to_openmm_simulation(
    integrator=openmm.LangevinMiddleIntegrator(
        300 * openmm.unit.kelvin,
        1 / openmm.unit.picosecond,
        2 * openmm.unit.femtosecond,
    )
)

simulation.minimizeEnergy()

# Add a reporter to record the structure every few steps
dcd_reporter = openmm.app.DCDReporter(file="rna.dcd", reportInterval=1000)
simulation.reporters.append(dcd_reporter)

simulation.context.setVelocitiesToTemperature(simulation.integrator.getTemperature())

In [ ]:
simulation.runForClockTime(1 * openmm.unit.minute)

In [ ]:
from utils import nglview_show_openmm

w = nglview_show_openmm(simulation.topology, "rna.dcd")
w.add_licorice(selection="not (HOH NA CL)")
w.add_spacefill(selection="HOH", opacity=0.05)
w.add_spacefill(selection="NA CL")
w.add_unitcell()
w

# Graph charges with NAGL

NAGL is a graph neural network toolkit designed for constructing, using, and distributing models that can compute partial charges from a molecular graph. NAGL charges are computed directly from the connectivity graph, are independent of conformation, are much faster than QC methods, and scale better to larger molecules. Currently released NAGL models are trained to reproduce AM1BCC charges for molecules that Sage is designed to support.

This notebook is YELLOW because:
- NAGL is pre-release and experimental

Linezolid is an reserve antibiotic for MDR Gram-positive infections

In [ ]:
from openff.toolkit import Molecule

linezolid = Molecule.from_smiles("O=C1O[C@@H](CNC(=O)C)CN1c3cc(F)c(N2CCOCC2)cc3")

linezolid.visualize(show_all_hydrogens=False)

In [ ]:
from openff.toolkit.utils.toolkits import ToolkitRegistry, toolkit_registry_manager, RDKitToolkitWrapper, BuiltInToolkitWrapper
from openff.toolkit.utils.nagl_wrapper import NAGLToolkitWrapper

In [ ]:
NAGLToolkitWrapper().supported_charge_methods

In [ ]:
# Do a "sacrificial" first run of nagl on methane so the model is compiled in this cell
Molecule.from_smiles("C").assign_partial_charges(
    "openff-gnn-am1bcc-0.1.0-rc.1.pt", toolkit_registry=NAGLToolkitWrapper()
)

## Compute NAGL and Antechamber charges

In [ ]:
from openff.toolkit.utils.nagl_wrapper import NAGLToolkitWrapper

with toolkit_registry_manager(ToolkitRegistry([NAGLToolkitWrapper, RDKitToolkitWrapper, BuiltInToolkitWrapper])):
    linezolid_nagl = Molecule(linezolid)
    linezolid_nagl.assign_partial_charges('openff-gnn-am1bcc-0.1.0-rc.1.pt')

In [ ]:
linezolid_nagl.partial_charges

In [ ]:
from openff.toolkit.utils.toolkits import AmberToolsToolkitWrapper

with toolkit_registry_manager(ToolkitRegistry([ AmberToolsToolkitWrapper, RDKitToolkitWrapper, BuiltInToolkitWrapper])):
    linezolid_antechamber = Molecule(linezolid)
    linezolid_antechamber.assign_partial_charges('AM1BCC')

In [ ]:
linezolid_antechamber.partial_charges

## Compare NAGL and Antechamber charges

In [ ]:
from matplotlib import pyplot

pyplot.scatter(linezolid_nagl.partial_charges.m, linezolid_antechamber.partial_charges.m)
pyplot.ylabel(f"reference charges ({linezolid_antechamber.partial_charges.u})")
pyplot.xlabel(f"nagl charges ({linezolid_nagl.partial_charges.u})")
pyplot.axline((0,0), slope=1, linestyle=":")

In [ ]:
charge_diffs = linezolid_nagl.partial_charges - linezolid_antechamber.partial_charges
pyplot.bar(range(len(charge_diffs)), charge_diffs.m)
pyplot.ylabel(f"charge difference ({charge_diffs.u})")
pyplot.xlabel("atom index")

In [ ]:
from utils import draw_molecule

draw_molecule(linezolid, atom_notes={i: f'{i}' for i in range(linezolid.n_atoms)})

# Custom substructure loading from PDB and NAGL charge assignment

This is a RED example showcasing early prototypes of functionality that we plan to make available in the coming year. We provide no guarantee of stability or correct operation, but this is an example of what may be to come.

This example will:
* Load a modified protein from PDB using `Topology.from_pdb`
* Create a librarycharge for just the modified residue and create the OpenMM system with ff14SB parameters

See also:
https://gist.github.com/Yoshanuikabundi/66007cb9966b1455a259baaf7cd7e7c3#file-ncaa_parametrization-ipynb

Why it's red:
- NAGL is experimental and pre-release
- Charging a whole protein with NAGL will not give the same charges as FF14SB
- Sage parameters have not been rigourously validated for use in non-canonical amino acids.

## Introduction

Here we have a PDB structure containing a protein with sequence "ACE-ALA5-DYE-ALA5-NME". The DYE residue is a cysteine labelled with a fluorophore via a maleimide "click" reaction.

In [ ]:
import nglview

w = nglview.show_file("inputs/3ip9_dye_solvated.pdb")
w.clear_representations()
w.add_representation("cartoon")
w.add_representation("licorice", selection="backbone or DYE")
w.add_representation("spacefill", selection="NA CL")
w.add_representation("spacefill", selection="HOH", opacity=0.05)
w

## Load a PDB with non-canonical amino acid


We're using OpenMM's PDBFile class under the hood for the initial loading. This leads to the following requirements in the input PDB:

* Standard residues must have canonical residue and atom names
* Modified residues must have elements explicitly listed in the last column
* The connectivity (but not bond order) of the modified residue must be explicitly specified using CONECT records (including peptide bonds to adjacent residues)

Let's get started on the workflow by making a representation of the non-canonical amino acid, which we'll use to teach our PDB loader about the new substructure. I'm labeling the connection points with [Fr] atoms.

In [ ]:
from openff.toolkit import Molecule, Topology

# Make unnatural AA as its own mol. Since a residue is a substructure, not a valid molecule,
# use something that's easy to recognize as a "cap". Here I use Fr since it makes one bond,
# we're super unlikely to see Fr anywhere else, and Sage doesn't support it, so if
# we make a mistake and it gets left behind we'll eventually get an error.
substructure_mol = Molecule.from_smiles(
    "C1=CC2=C(C=C1N3C(=O)C[C@H](SC[C@H](N[Fr])C(=O)[Fr])C3=O)C(=O)O[C@]24C5=C(C=C(C=C5)O)OC6=C4C=CC(=C6)O"
)
substructure_mol.visualize(show_all_hydrogens=False)

In [ ]:
# Label the atoms with whether they're leaving
for atom in substructure_mol.atoms:
    if atom.symbol == "Fr":
        atom.metadata["substructure_atom"] = False
    else:
        atom.metadata["substructure_atom"] = True

In [ ]:
top = Topology.from_pdb(
    "inputs/3ip9_dye_solvated.pdb",
    # _additional_substructures is a PROTOTYPE.
    # Its behavior and input type are likely to change.
    _additional_substructures=[substructure_mol],
)

In [ ]:
w = top.visualize()
w.clear_representations()
w.add_representation("cartoon")
w.add_representation("licorice", selection="not (HOH NA CL)")
w.add_representation("spacefill", selection="NA CL")
w.add_representation("spacefill", selection="HOH", opacity=0.05)
w

## Compute NAGL partial charges

Here, for simplicity, we simply compute NAGL charges for the entire peptide. A more rigorous approach might combine NAGL or Antechamber charges for the dye with Amber library charges for the other residues, as was done in a previous workshop:

https://www.youtube.com/watch?v=lZ4UUgHQpWg

In [ ]:
from openff.nagl_models import list_available_nagl_models

[path.name for path in list_available_nagl_models()]

In [ ]:
protein = top.molecule(0)

In [ ]:
from openff.toolkit.utils.nagl_wrapper import NAGLToolkitWrapper

protein.assign_partial_charges(
    "openff-gnn-am1bcc-0.1.0-rc.1.pt", toolkit_registry=NAGLToolkitWrapper()
)
print(protein.partial_charges)

### Comparing NAGL charges to Amber library charges

NAGL charges are quite different to the charges assigned by Amber, as they are based on AM1-BCC rather than RESP calculations. These methods are considered to produce "compatible" charges for intermolecular interactions, but this will have an effect on the Amber force field's intramolecular interactions. A future OpenFF protein force field is being developed with charges much more similar to NAGL, and NAGL charges will later be optimized in concert with OpenFF forcce fields.

Library charges can only be assigned to a molecule if all atoms in that molecule can be assigned one; they cannot be mixed within a molecule with other charge methods. We can define a dummy library charge for generic atoms to allow Amber library charges to be assigned to the remainder of the protein.

In [ ]:
from io import StringIO

from openff.interchange.smirnoff import SMIRNOFFElectrostaticsCollection
from openff.toolkit import ForceField

# Construct a force field that assigns any atom a charge of 0
generic_librarycharge = """<?xml version="1.0" encoding="utf-8"?>
<SMIRNOFF version="0.3" aromaticity_model="OEAroModel_MDL">        
  <LibraryCharges version="0.3">
    <LibraryCharge smirks="[*:1]" charge1="0.0 * elementary_charge" id="generic_zero"></LibraryCharge>
  </LibraryCharges>
</SMIRNOFF>
"""

# Combine the above library charge with ff14SB
# General force field first so that Amber wins any conflicts!
with StringIO(generic_librarycharge) as generic_librarycharge_offxml:
    amber_charges = ForceField(
        generic_librarycharge_offxml,
        "ff14sb_off_impropers_0.0.4.offxml",
    )

# Use the above force field to force Amber library charges on all atoms
# Construct an Electrostatics collection directly to avoid issues with
# missing bonded or vdW parameters
librarycharges_collection = SMIRNOFFElectrostaticsCollection.create(
    parameter_handler=[
        amber_charges["Electrostatics"],
        amber_charges["LibraryCharges"],
    ],
    topology=protein.to_topology(),
)

Identify the charges in parts of the protein other than the dye (where the Amber charges are undefined).

In [ ]:
from openff.units import unit

dye_atom_indices = [
    atom.molecule_atom_index
    for atom in [res for res in protein.residues if res.residue_name == "DYE"][0].atoms
]

nagl_partial_charges = [
    q.m_as(unit.elementary_charge)
    for i, q in enumerate(protein.partial_charges)
    if i not in dye_atom_indices
] * unit.elementary_charge
amber_partial_charges = [
    q.m_as(unit.elementary_charge)
    for i, q in enumerate(librarycharges_collection.charges.values())
    if i not in dye_atom_indices
] * unit.elementary_charge

Plot the NAGL charges against Amber charges.

In [ ]:
from matplotlib import pyplot

pyplot.scatter(
    nagl_partial_charges.m,
    amber_partial_charges.m,
    c=range(len(nagl_partial_charges)),
    marker=".",
)
pyplot.xlabel(f"nagl charges ({nagl_partial_charges.u})")
pyplot.ylabel(f"reference charges ({amber_partial_charges.u})")
pyplot.axline((0, 0), slope=1, linestyle=":")

If you're interested, we can break down assigned charges by atom and residue name:

In [ ]:
# # Note: amber charges vary when multiple residue types are assigned the same residue name; 
# # eg, with different protonation states or for terminal residues

# from collections import defaultdict
# from functools import partial

# # Break charges down by atom and residue name
# nagl_charges_by_atomtype = defaultdict(partial(defaultdict, list))
# ff14sb_charges_by_atomtype = defaultdict(partial(defaultdict, list))
# for nagl_charge, ff14sb_charge, atom in zip(
#     protein.partial_charges, librarycharges_collection.charges.values(), protein.atoms
# ):
#     nagl_charges_by_atomtype[atom.metadata["residue_name"]][atom.name].append(
#         nagl_charge.m_as(unit.elementary_charge)
#     )
#     ff14sb_charges_by_atomtype[atom.metadata["residue_name"]][atom.name].append(
#         ff14sb_charge.m_as(unit.elementary_charge)
#     )

# # Create a plot for the 21 different residue names (20 canonical AAs plus DYE)
# fig, axs = pyplot.subplots(
#     nrows=7,
#     ncols=3,
#     figsize=(20, 30),
#     sharey=True,
#     dpi=180,
#     constrained_layout=True,
#     gridspec_kw={"hspace": 0.1, "wspace": 0.05},
# )

# # Plot the charges from both force fields
# for (resname, by_atomname), ax in zip(nagl_charges_by_atomtype.items(), axs.flatten()):
#     atomnames, nagl_charges = zip(*by_atomname.items())
#     amber_charges = [
#         ff14sb_charges_by_atomtype[resname][atomname] for atomname in atomnames
#     ]
#     boxplot = ax.boxplot(
#         nagl_charges,
#         showfliers=False,
#         labels=atomnames,
#         medianprops={"color": "black"},
#         positions=range(len(nagl_charges)),
#     )
#     nagl_events = ax.eventplot(nagl_charges, orientation="vertical", color="orange")
#     amber_events = ax.eventplot(amber_charges, orientation="vertical")
#     ax.set_title(f"{resname} (n={len(nagl_charges[0])})")
#     ax.set_xmargin(0)

#     if resname == "MET":
#         amber_events[0].set_label("Amber charges")
#         nagl_events[0].set_label("NAGL charges")
#         boxplot["boxes"][0].set_label("NAGL charges box plot")
#         ax.legend(loc="lower right")
#         ax.set_ylabel("Partial Charge (e-)")

## Parametrize and simulate the peptide with Sage, Amber ff14sb, and NAGL charges

In [ ]:
from openff.toolkit import ForceField

sage_ff14sb = ForceField("openff-2.2.0.offxml", "ff14sb_off_impropers_0.0.4.offxml")

In [ ]:
ic = sage_ff14sb.create_interchange(top, charge_from_molecules=[protein])

In [ ]:
import openmm

simulation = ic.to_openmm_simulation(
    openmm.LangevinIntegrator(
        300 * openmm.unit.kelvin,
        1 / openmm.unit.picosecond,
        2 * openmm.unit.femtoseconds,
    )
)

# Energy minimize
simulation.minimizeEnergy()

# Add a reporter to record the structure every 1000 steps
dcd_reporter = openmm.app.DCDReporter("ncaa.dcd", 1000)
simulation.reporters.append(dcd_reporter)
simulation.context.setVelocitiesToTemperature(simulation.integrator.getTemperature())

In [ ]:
simulation.runForClockTime(1 * openmm.unit.minute)

In [ ]:
from utils import nglview_show_openmm

w = nglview_show_openmm(simulation.topology, "ncaa.dcd")
w.clear_representations()
w.add_representation("cartoon")
w.add_representation("licorice", selection="not (HOH NA CL)")
w.add_representation("spacefill", selection="NA CL")
w.add_representation("spacefill", selection="HOH", opacity=0.05)
w